[View in Colaboratory](https://colab.research.google.com/github/pgurazada/ml-projects/blob/master/2018_06_7_keras_regression_grid.ipynb)

In [1]:
import keras

Using TensorFlow backend.


In [37]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) =  boston_housing.load_data()

57344/57026 [==============================] - 0s 5us/step


In [0]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import RMSprop
from keras.losses import mean_squared_error
from keras.metrics import mse

from keras import backend as K

from keras.wrappers.scikit_learn import KerasRegressor

In [0]:
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV

In [0]:
def build(network_type=Sequential,
          nb_initial_layer=64,
          dense_layer_list=[64],
          nb_final_layer=1,
          learning_rate=0.001):
    
    model = network_type()
    
    model.add(Dense(nb_initial_layer, input_shape=(train_data.shape[1],)))
    model.add(Activation('relu'))
    
    for nb_units in dense_layer_list:
        model.add(Dense(nb_units))
        model.add(Activation('relu'))
    
    model.add(Dense(nb_final_layer)) # No activation for the final layer, since this is a regression problem
    
    model.compile(optimizer=RMSprop(lr=learning_rate),
                  loss=mean_squared_error,
                  metrics=[mse])
    
    return model

In [0]:
K.clear_session()

In [0]:
model = KerasRegressor(build, verbose=0)

In [0]:
pgrid = {'epochs' : [25],
         'batch_size' : [100],
         'learning_rate' : [0.01, 0.001],
         'nb_initial_layer' : [48, 64, 128],
         'dense_layer_list' : [[64], [64, 64], [64, 32, 32], [32], [24]]}

In [0]:
result_grid = RandomizedSearchCV(estimator=model,
                                 param_distributions=pgrid, 
                                 n_iter=5,
                                 scoring='neg_mean_squared_error')

In [35]:
%%time
result_grid.fit(train_data, train_targets)

CPU times: user 33.9 s, sys: 1.44 s, total: 35.4 s
Wall time: 33.2 s


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f9153255f98>,
          fit_params=None, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'epochs': [25], 'batch_size': [100], 'learning_rate': [0.01, 0.001], 'nb_initial_layer': [48, 64, 128], 'dense_layer_list': [[64], [64, 64], [64, 32, 32], [32], [24]]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=0)

In [36]:
result_grid.best_params_

{'batch_size': 100,
 'dense_layer_list': [32],
 'epochs': 25,
 'learning_rate': 0.01,
 'nb_initial_layer': 64}

In [38]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0

Setting up unattended-upgrades (0.98ubuntu1.1) ...

Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17)

In [0]:
from google.colab import auth
auth.authenticate_user()

In [40]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!mkdir -p /content/drive/data/

In [0]:
!ls /content/drive/data/

In [44]:
!ls

datalab  drive


In [0]:
!cd drive

In [46]:
!ls

datalab  drive
